In [ ]:
!pip install scikit-learn

In [2]:
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Functies


De find_sim_single_summary functie met gegeven een text en de samenvatting van deze tekst, returned een array met daarin per zin in de samenvatting, de index van de zin in de text die er het meest mee overeen komt.

In [3]:
def find_sim_single_summary(summary_sentence_embed, doc_emedding):
    cos_sim_mat = cosine_similarity(doc_emedding, summary_sentence_embed)
    idx_arr = np.argmax(cos_sim_mat, axis=0)
    
    return idx_arr

De label_sent_in_summary functie met gegeven een lijst met teksten en een lijst met de samenvattingen van die teksten, returned deze functie:

*   idx_list : Een lijst van arrays, waarvan elke array een samenvatting voorstelt met per zin in de samenvatting de index van de overeenkomstige zin in de tekst(waarvan deze samenvatting is).
*   labels : Een lijst van arrays, waarvan elke array een tekst voorstelt met per zin in deze tekst een boolean waarde of dat deze zin wel of niet in de samenvatting is gebruikt.



In [4]:
def label_sent_in_summary(s_text, s_summary):
    doc_num = s_text.shape[0]
    
    #initialize zeros
    labels = [np.zeros(doc.shape[0]) for doc in s_text.tolist()] 
    
    #calc idx for most similar
    idx_list = [np.sort(find_sim_single_summary(s_summary[j], s_text[j])) for j in range(doc_num)]
      
    for j in range(doc_num):
        labels[j][idx_list[j]]= 1 
    
    return idx_list, labels

# Script


Het definiëren van input en output file, de input file lezen en in een dataframe plaatsen.  
En hierna de tekst embeddings en samenvatting embeddings gebruiken om de labels en index lijst te genereren.

In [5]:
input_file = '../data/df_processed.pickle'
output_file = '../data/df_processed_label.pickle'

df = pd.read_pickle(input_file)

#get index list and target labels
idx_list, labels = label_sent_in_summary(df.text_embedding, df.sum_embedding)

Deze labels en index list bij in de dataframe zetten.  
En vervolgens deze dataframe omvormen naar een pickle bestand om weg te schrijven.

In [6]:
# wrap in dataframe
df['labels'] = labels   # Een boolean per zin in de text, die weergeeft of deze relevant is ofniet
df['labels_idx_list'] = idx_list    # per zin in de summarie, de index van de zin in de text die er het meest mee overeen komt

# save to pickle
with open(output_file, 'wb') as handle:                                     
    pickle.dump(df, handle)